In [18]:
import numpy as np
from numba import vectorize
from numpy import linalg as la

In [44]:
SPEED_OF_LIGHT = 299792458.0 
permittivity = 15.0
conductivity = 3e-2

In [4]:
@vectorize
def lin2db(value_linear):
    return 10 * np.log10(value_linear) if value_linear >= 1e-15 else -np.inf

In [5]:
def dipole_rp(*, azimuth, **kwargs):
    """
    Returns dipole directional gain
    :param azimuth:
    :return:
    """
    c = np.cos(azimuth)
    s = np.sin(azimuth)
    if c > 1e-9:
        return np.abs(np.cos(np.pi / 2 * s) / c)
    else:
        return 0.0

In [6]:
def _reflection_c_parallel(grazing_angle, permittivity, conductivity, wavelen):
    eta = permittivity - 60j * wavelen * conductivity
    c = np.cos(grazing_angle)
    return (eta - c ** 2) ** 0.5


def _reflection_c_perpendicular(grazing_angle, permittivity, conductivity, wavelen):
    eta = permittivity - 60j * wavelen * conductivity
    c = np.cos(grazing_angle)
    return (eta - c ** 2) ** 0.5 / eta


# noinspection PyUnusedLocal
def reflection_constant(*, grazing_angle, polarization, permittivity, conductivity, wavelen, **kwargs):
    return -1.0 + 0.j


# noinspection PyUnusedLocal
# @vectorize
def reflection(*, grazing_angle, polarization, permittivity, conductivity, wavelen, **kwargs):
    """
    Computes reflection coefficient from conducting surface with defined grazing angle and supported relative
    permittivity and conductivity of the surface. In order to set type of wave polarization, polarization
    parameter is specified. For parallel to surface polarized wave polarization should set to 1, for perpendicular -
    to 0; for circular - to 0.5. For different type of elliptic polarization use other value in the range of 0..1
    :param grazing_angle: an angle between normal to surface and wave vector
    :param polarization: determine the type of polarization of the grazing wave
    :param permittivity: the relative_permittivity of two media divided by the surface
    :param conductivity: the conductivity of the surface
    :param wavelen: the wave length of the grazing wave
    :return: the reflection coefficient for specified parameters and the given grazing angle
    """
    s = np.sin(grazing_angle)

    if polarization < 0 or polarization > 1:
        return float('nan')

    if polarization != 0:
        c_parallel = _reflection_c_parallel(grazing_angle, permittivity, conductivity, wavelen)
        reflection_parallel = (s - c_parallel) / (s + c_parallel)
    else:
        reflection_parallel = 0.j

    if polarization != 1:
        c_perpendicular = _reflection_c_perpendicular(grazing_angle, permittivity, conductivity, wavelen)
        reflection_perpendicular = (s - c_perpendicular) / (s + c_perpendicular)
    else:
        reflection_perpendicular = 0.j

    return polarization * reflection_parallel + (1 - polarization) * reflection_perpendicular

In [57]:
def two_ray_path_loss_3d(tx_pos, tx_dir_theta, rx_pos, rx_dir_theta, polarization):
    """
    Computes free space signal attenuation between the transmitter and the receiver in linear scale.
    :param wavelen: a wavelen of signal carrier
    :param time: Time passed from the start of reception
    :param ground_reflection: a function to compute a complex-valued reflection coefficient
    :param tx_velocity: the velocity of the transmitter
    :param tx_dir_phi: the vector pointed the direction with tilt angle equals 0 of the transmitter antenna.
    :param tx_dir_theta: the vector pointed the direction with azimuth angle equals 0 of the transmitter antenna.
    :param tx_pos: a current position of the transmitter.
    :param tx_rp: a radiation pattern of the transmitter
    :param rx_velocity: the velocity of the receiver
    :param rx_dir_phi: the vector pointed the direction with tilt angle equals 0 of the transmitter antenna.
    :param rx_dir_theta: the vector pointed the direction with azimuth angle equals 0 of the transmitter antenna.
    :param rx_pos: a current position of the receiver
    :param rx_rp: a radiation pattern of the receiver
    :return: free space path loss in linear scale
    """

    permittivity = 15.0
    conductivity = 3e-2
    wavelen=SPEED_OF_LIGHT/860e6

    def dipole_rp(*, azimuth, **kwargs):  
        """
        Returns dipole directional gain
        :param azimuth:
        :return:
        """
        c = np.cos(azimuth)
        s = np.sin(azimuth)
        if c > 1e-9:
            return np.abs(np.cos(np.pi / 2 * s) / c)
        else:
            return 0.0

    wavelen = SPEED_OF_LIGHT/860e6,
    # LoS - Line-of-Sight, NLoS - Non-Line-of-Sight

    # Ray geometry computation
    ground_normal = np.array([0, 0, 1])
    rx_pos_refl = np.array([rx_pos[0], rx_pos[1], -rx_pos[2]])  # Reflect RX relatively the ground

    d0_vector = rx_pos - tx_pos            # LoS ray vector
    d1_vector = rx_pos_refl - tx_pos       # NLoS ray vector
    d0 = la.norm(d0_vector)                # LoS ray length
    d1 = la.norm(d1_vector)                # NLoS ray length
    d0_vector_tx_n = d0_vector / d0        # LoS ray vector normalized
    d0_vector_rx_n = -d0_vector_tx_n
    d1_vector_tx_n = d1_vector / d1        # NLoS ray vector normalized
    d1_vector_rx_n = np.array([-d1_vector_tx_n[0], -d1_vector_tx_n[1], d1_vector_tx_n[2]])

    # Azimuth and tilt angle computation for computation of attenuation
    # caused by deflection from polar direction
    tx_azimuth_0 = np.arccos(np.dot(d0_vector_tx_n, tx_dir_theta))
    rx_azimuth_0 = np.arccos(np.dot(d0_vector_rx_n, rx_dir_theta))
    tx_azimuth_1 = np.arccos(np.dot(d1_vector_tx_n, tx_dir_theta))
    rx_azimuth_1 = np.arccos(np.dot(d1_vector_rx_n, rx_dir_theta))

    # A grazing angle of NLoS ray for computation of reflection coefficient
    grazing_angle = np.arccos(-1*np.dot(d1_vector_rx_n, ground_normal))

    #relative_velocity = rx_velocity - tx_velocity
    #velocity_pr_0 = np.dot(d0_vector_tx_n, relative_velocity)
    #velocity_pr_1 = np.dot(d1_vector_tx_n, relative_velocity)

    # Attenuation caused by radiation pattern


    g0 = (dipole_rp(azimuth=tx_azimuth_0) * dipole_rp(azimuth=rx_azimuth_0))

    g1 = (dipole_rp(azimuth=tx_azimuth_1) * dipole_rp(azimuth=rx_azimuth_1))

    # Attenuation due to reflections (reflection coefficient) computation

    def _reflection_c_parallel(grazing_angle, permittivity, conductivity, wavelen):
        #eta = permittivity - 60j * wavelen * conductivity
        eta = 15 - 60j * SPEED_OF_LIGHT * 3e-2 / 860e6
        c = np.cos(grazing_angle)
        return (eta - c ** 2) ** 0.5


    def _reflection_c_perpendicular(grazing_angle, permittivity, conductivity, wavelen):
        #eta = permittivity - 60j * wavelen * conductivity
        eta = 15 - 60j * SPEED_OF_LIGHT * 3e-2 / 860e6
        c = np.cos(grazing_angle)
        return (eta - c ** 2) ** 0.5 / eta


    # noinspection PyUnusedLocal
    def reflection_constant(*, grazing_angle, polarization, permittivity, conductivity, wavelen, **kwargs):
        return -1.0 + 0.j


    # noinspection PyUnusedLocal
    # @vectorize
    def reflection(grazing_angle, polarization, permittivity, conductivity, wavelen):
        """
        Computes reflection coefficient from conducting surface with defined grazing angle and supported relative
        permittivity and conductivity of the surface. In order to set type of wave polarization, polarization
        parameter is specified. For parallel to surface polarized wave polarization should set to 1, for perpendicular -
        to 0; for circular - to 0.5. For different type of elliptic polarization use other value in the range of 0..1
        :param grazing_angle: an angle between normal to surface and wave vector
        :param polarization: determine the type of polarization of the grazing wave
        :param permittivity: the relative_permittivity of two media divided by the surface
        :param conductivity: the conductivity of the surface
        :param wavelen: the wave length of the grazing wave
        :return: the reflection coefficient for specified parameters and the given grazing angle
        """
        s = np.sin(grazing_angle)

        if polarization < 0 or polarization > 1:
            return float('nan')

        if polarization != 0:
            c_parallel = _reflection_c_parallel(grazing_angle, permittivity, conductivity, wavelen)
            reflection_parallel = (s - c_parallel) / (s + c_parallel)
        else:
            reflection_parallel = 0.j

        if polarization != 1:
            c_perpendicular = _reflection_c_perpendicular(grazing_angle, permittivity, conductivity, wavelen)
            reflection_perpendicular = (s - c_perpendicular) / (s + c_perpendicular)
        else:
            reflection_perpendicular = 0.j

        return polarization * reflection_parallel + (1 - polarization) * reflection_perpendicular

    r1 = reflection(grazing_angle=grazing_angle, wavelen=wavelen, polarization=polarization, permittivity = 15.0, conductivity = 3e-2)


    #k = 2 * np.pi / wavelen
    k = 18.02426718878446
    return (0.5/k)**2 * np.absolute(   g0/d0*np.exp(-1j*k*d0) +
                                    r1*g1/d1*np.exp(-1j*k*d1))**2


In [ ]:
two_ray_path_loss_3d()

In [51]:
def get_path_loss(self, sender, receiver):

    sender_velocity = np.array((0, 0, 0))
    receiver_velocity = np.array((0, 0, 0))


    # Compute path loss in linear scale
    pl = pyradise.two_ray_path_loss_3d(
        time=0,
        wavelen=SPEED_OF_LIGHT/860e6,
        tx_pos=tx_antenna.position,
        tx_dir_theta=tx_antenna.dir_forward,
        tx_dir_phi=tx_antenna.dir_right,
        tx_rp=tx_antenna.rp,
        tx_velocity=sender_velocity,
        rx_pos=rx_antenna.position,
        rx_dir_theta=rx_antenna.dir_forward,
        rx_dir_phi=rx_antenna.dir_right,
        rx_rp=rx_antenna.rp,
        rx_velocity=receiver_velocity,
        ground_reflection=self.ground_reflection,
        permittivity=self.ground_permittivity,
        conductivity=self.ground_conductivity,
        polarization=tx_antenna.polarization)


    return pyradise.lin2db(pl)

def get_rx_power(self, sender, receiver, path_loss=None, tx_power=None):

    if path_loss is None:
        path_loss = self.get_path_loss(sender=sender, receiver=receiver)

    polarization_loss = receiver.node.antenna.get_polarization_loss(
        sender.node.antenna.polarization)

    rx_power = (tx_power + sender.modulation_loss +
                sender.node.antenna.cable_loss + sender.node.antenna.gain +
                path_loss + receiver.node.antenna.gain + polarization_loss)

    return rx_power

In [37]:
x_reader = 1.0
y_reader = 0.0
z_reader = 5.0
a = np.pi / 4
forward_dir_reader = (-np.sin(a), 0, -np.cos(a))
right_dir_reader = (0, -1, 0)
position_reader = (-x_reader, y_reader, z_reader)
polapolarization_reader = 0.5

x_tag = 10.0
y_tag = 0.0
z_tag = 0.5

#tag

polapolarization_tag = 1

In [70]:
two_ray_path_loss_3d(position_reader, forward_dir_reader,
                         np.array([9, 0, 0.5]), np.array([1,0,0]), 0.5)
                         

0.0

In [56]:
k = 2 * np.pi / wavelen
k

18.02426718878446

In [39]:
60j+1

(1+60j)